Parsing and head automata
----

Decoding parse trees with head automata for consecutive siblings

In [1]:
import numpy as np 
import ad3
from ad3.extensions import PFactorTree, PFactorHeadAutomaton

### Initialize variables and set some random scores

The 2d array `scores` has the score, or log potential, for an arc from token `i` to `j` in each cell `(i, j)`

To make the result more predictable, we increase the scores for `(0, 2)` and from `2` to the other tokens.

In [2]:
g = ad3.factor_graph.PFactorGraph()
arc_variables = []
arcs = []

num_tokens = 5  # including root

# create indices and scores from every node to all others
scores = np.random.normal(0, 1, [num_tokens, num_tokens])

# root is never a modifier
scores[:, 0] -= 1000

scores[0, 2] += 5
scores[2, 1:] += 5

### Add the tree factor to the graph

The tree factor enforces that the solution is a valid dependency tree. Basically, it enforces the following constraints:

* There are no cycles
* The first token -- the root -- has exactly one outgoing arc and no incoming arcs

In [3]:
index_arcs = np.full([num_tokens, num_tokens], -1, np.int)

# suppose the arc 0 -> 1 has been pruned
filtered_arcs = set([(0, 1)])

for h in range(num_tokens):
    for m in range(1, num_tokens):
        if h == m or (h, m) in filtered_arcs:
            continue
            
        arcs.append((h, m))
        arc_var = g.create_binary_variable()
        arc_var.set_log_potential(scores[h, m])
        index_arcs[h, m] = len(arc_variables)
        arc_variables.append(arc_var)

tree = PFactorTree()
g.declare_factor(tree, arc_variables)
tree.initialize(num_tokens, arcs)

Show the values of the scores for each arc

In [4]:
for arc in arcs:
    h, m = arc
    score = scores[h, m]
    print(arc, score)

(0, 2) 5.7760569027915025
(0, 3) 1.6444635700822923
(0, 4) -0.14821278718845635
(1, 2) 0.294288915084736
(1, 3) 0.732306242916495
(1, 4) -0.4432597952457511
(2, 1) 3.132133121112913
(2, 3) 5.495615776907177
(2, 4) 5.764295209790005
(3, 1) 0.5600640564813785
(3, 2) -0.04908613842503491
(3, 4) 0.5436349044985233
(4, 1) 0.7770996283822647
(4, 2) 1.0941617743846943
(4, 3) 1.244807848924485


Solve for the exact solution

In [ ]:
value, posteriors, additional_posteriors, status = g.solve_exact_map_ad3()
print(value)
print(posteriors)
print(additional_posteriors)
print(status)

In [5]:
def show_selected_items(posteriors, arcs, show_posterior=False):
    for p, arc in zip(posteriors, arcs):
        if p ** 2 <= 0.0001:
            continue
        if show_posterior:
            print(arc, p)
        else:
            print(arc)


In [ ]:
show_selected_items(posteriors, arcs)

Now, let's add next sibling features and head automata

In [6]:
def create_empty_lists(n):
    """
    Create a list with n empty lists
    """
    return [[] for _ in range(n)]


# create all possible sibling combinations considering non-pruned arcs
sibling_parts_left = create_empty_lists(num_tokens)
sibling_parts_right = create_empty_lists(num_tokens)

for h in range(num_tokens):
    # right side
    for m in range(h, num_tokens):
        if h != m and index_arcs[h, m] == -1:
            continue
            
        for s in range(m + 1, num_tokens + 1):
            # s == num_tokens signals that m is the rightmost child
            if s < num_tokens and index_arcs[h, s] == -1:
                continue
            
            sibling_part = (h, m, s)
            sibling_parts_right[h].append(sibling_part)
    
    for m in range(h, 0, -1):        
        if h != m and (h, m) in filtered_arcs:
            continue
        
        for s in range(m - 1, -1, -1): 
            # s == 0 signals that m is the leftmost child
            if s > 0 and (h, s) in filtered_arcs:
                continue
            
            sibling_part = (h, m, s)
            sibling_parts_left[h].append(sibling_part)

# create random scores for each sibling part
# (we can't create an array here because each head has a different number of sibling parts)
sibling_scores_left = [np.random.normal(size=len(siblings)) 
                       for siblings in sibling_parts_left]
sibling_scores_right = [np.random.normal(size=len(siblings)) 
                        for siblings in sibling_parts_right]

Check that there is a score for each sibling part either in right and left hand sides

In [7]:
def print_siblings(sibling_parts, sibling_scores):
    for i, (head_siblings, scores) in enumerate(zip(sibling_parts, sibling_scores)):
        print('Head', i, ': ', len(head_siblings), 'sibling parts and', len(scores), 'scores')
        print('--------------------')
        for sibling_part, part_score in zip(head_siblings, scores):
            print(sibling_part, part_score)
        print()

In [8]:
print('Left siblings:')
print_siblings(sibling_parts_left, sibling_scores_left)
print()
print_siblings(sibling_parts_right, sibling_scores_right)

Left siblings:
Head 0 :  0 sibling parts and 0 scores
--------------------

Head 1 :  1 sibling parts and 1 scores
--------------------
(1, 1, 0) 0.5204535461879003

Head 2 :  3 sibling parts and 3 scores
--------------------
(2, 2, 1) -0.33601552498299153
(2, 2, 0) 0.25837323718400185
(2, 1, 0) -1.5033635205127176

Head 3 :  6 sibling parts and 6 scores
--------------------
(3, 3, 2) -0.487739206384911
(3, 3, 1) -1.1361315057744434
(3, 3, 0) 0.2512526124908858
(3, 2, 1) 0.18674125561794863
(3, 2, 0) -0.14520171412449456
(3, 1, 0) -1.420999171516399

Head 4 :  10 sibling parts and 10 scores
--------------------
(4, 4, 3) -0.4338401003126975
(4, 4, 2) 0.4199875922663737
(4, 4, 1) -1.0052312280137718
(4, 4, 0) 0.8608856650066535
(4, 3, 2) 1.6130727254977213
(4, 3, 1) -0.18884877779389317
(4, 3, 0) -1.5427211687240174
(4, 2, 1) 0.9847884574873762
(4, 2, 0) 1.3404670864346517
(4, 1, 0) -0.09994757120302905


Head 0 :  10 sibling parts and 10 scores
--------------------
(0, 0, 2) -0.2812718

Include sibling factors in the graph

In [9]:
additionals = []

# there is a factor for each head and side combination
for h in range(num_tokens):
    
    # right hand size
    
    # local_vars indicates which variables are constrained by the factor
    local_vars = []
    local_siblings = []
    additional_scores = []
    local_arcs = []
    
    for m in range(h + 1, num_tokens):
        index = index_arcs[h, m]
        if index < 0:
            continue

        local_vars.append(arc_variables[index])
        local_arcs.append((h, m))
        
    factor = PFactorHeadAutomaton()
    
    # important: first declare the factor in the graph, then initialize
    # it may seems counter-intuitive but breaks otherwise
    
    # owned_by_graph must be True so that the variables don't get garbage collected
    g.declare_factor(factor, local_vars, owned_by_graph=True)
    factor.initialize(local_arcs, sibling_parts_right[h], validate=False)
    factor.set_additional_log_potentials(sibling_scores_right[h])
    additionals.extend(sibling_parts_right[h])
    
    # left hand size
    if h == 0:
        # root has no children to the left
        continue
        
    local_vars = []
    local_siblings = []
    additional_scores = []
    local_arcs = []
    for m in range(h, 0, -1):
        index = index_arcs[h, m]
        if index < 0:
            continue
            
        local_vars.append(arc_variables[index])
        local_arcs.append((h, m))
        
    factor = PFactorHeadAutomaton()
    
    # important: first declare the factor in the graph, then initialize
    # it may seems counter-intuitive but breaks otherwise
    g.declare_factor(factor, local_vars, owned_by_graph=True)
    factor.initialize(local_arcs, sibling_parts_left[h], validate=False)
    factor.set_additional_log_potentials(sibling_scores_left[h])
    additionals.extend(sibling_parts_left[h])

Solve again, now with the siblings

In [10]:
value, posteriors, additional_posteriors, status = g.solve_exact_map_ad3()
print(value)
print(posteriors)
print(additional_posteriors)
print(status)

20.98441602319053
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0


In [11]:
show_selected_items(posteriors, arcs)

(0, 2)
(2, 1)
(2, 3)
(2, 4)


In [12]:
show_selected_items(additional_posteriors, additionals)

(0, 0, 2)
(0, 2, 5)
(1, 1, 5)
(1, 1, 0)
(2, 2, 3)
(2, 3, 4)
(2, 4, 5)
(2, 2, 1)
(2, 1, 0)
(3, 3, 5)
(3, 3, 0)
(4, 4, 5)
(4, 4, 0)
